<a href="https://colab.research.google.com/github/jnels4/nfl_predictions/blob/john/NFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-06 23:16:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.24MB/s    in 0.2s    

2022-09-06 23:16:42 (6.24 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NFL-Prediction").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://nflpredictionsdataviz.s3.amazonaws.com/spreadspoke_scores.csv"
spark.sparkContext.addFile(url)
NFL_Guru_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("spreadspoke_scores.csv"), sep=",", header=True, inferSchema=True)
NFL_Guru_df.show()

+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|          team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/2/1966|           1966|            1|           false|     Miami Dolphins|        14|        23|     Oakland Raiders|            null|           null

In [5]:
NFL_Guru_df.printSchema()

root
 |-- schedule_date: string (nullable = true)
 |-- schedule_season: integer (nullable = true)
 |-- schedule_week: string (nullable = true)
 |-- schedule_playoff: boolean (nullable = true)
 |-- team_home: string (nullable = true)
 |-- score_home: integer (nullable = true)
 |-- score_away: integer (nullable = true)
 |-- team_away: string (nullable = true)
 |-- team_favorite_id: string (nullable = true)
 |-- spread_favorite: double (nullable = true)
 |-- over_under_line: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- stadium_neutral: boolean (nullable = true)
 |-- weather_temperature: integer (nullable = true)
 |-- weather_wind_mph: integer (nullable = true)
 |-- weather_humidity: integer (nullable = true)
 |-- weather_detail: string (nullable = true)



In [6]:
NFL_guru_current = NFL_Guru_df.where(NFL_Guru_df.schedule_season>1978)
NFL_guru_current.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|              TB|           

In [14]:
NFL_guru_jetsbills = NFL_guru_current.filter(((NFL_guru_current.team_home == 'New York Jets') & (NFL_guru_current.team_away == 'Buffalo Bills')) | ((NFL_guru_current.team_home == 'Buffalo Bills') & (NFL_guru_current.team_away == 'New York Jets')))
NFL_guru_jetsbills.show()

+-------------+---------------+-------------+----------------+-------------+----------+----------+-------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|    team_home|score_home|score_away|    team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|
+-------------+---------------+-------------+----------------+-------------+----------+----------+-------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+
|    9/23/1979|           1979|            4|           false|Buffalo Bills|        46|        31|New York Jets|             BUF|           -3.0|             42|Ralph Wilson Stadium|          fals

In [7]:
from pyspark.sql import functions as F

NFL_guru_wins = NFL_guru_current.withColumn('home_win',
                                            F.when((F.col('score_home')>F.col('score_away')), 2)\
                                            .when((F.col('score_home')<F.col('score_away')), 1).otherwise(1)                                   
                                            )
NFL_guru_wins.show()

+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|schedule_date|schedule_season|schedule_week|schedule_playoff|           team_home|score_home|score_away|           team_away|team_favorite_id|spread_favorite|over_under_line|             stadium|stadium_neutral|weather_temperature|weather_wind_mph|weather_humidity|weather_detail|home_win|
+-------------+---------------+-------------+----------------+--------------------+----------+----------+--------------------+----------------+---------------+---------------+--------------------+---------------+-------------------+----------------+----------------+--------------+--------+
|     9/1/1979|           1979|            1|           false|Tampa Bay Buccaneers|        31|        16|       Detroit Lions| 

In [16]:
NFL_guru_jetsbills = NFL_guru_jetsbills.withColumn('home_win',
                                            F.when((F.col('score_home')>F.col('score_away')), 2)\
                                            .when((F.col('score_home')<F.col('score_away')), 1).otherwise(1)                                   
                                            )

In [8]:
cols = ('schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff', 'team_favorite_id', 'stadium_neutral','weather_humidity', 'over_under_line')
NFL_short = NFL_guru_wins.drop(*cols)
NFL_short.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|          null|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|          null|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|          null|       2|
|      Denver Br

In [19]:
NFL_guru_jetsbills = NFL_guru_jetsbills.drop(*cols)
NFL_guru_jetsbills.show()

+-------------+----------+----------+-------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|    team_home|score_home|score_away|    team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+-------------+----------+----------+-------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Buffalo Bills|        46|        31|New York Jets|           -3.0|Ralph Wilson Stadium|                 50|               9|          null|       2|
|New York Jets|        12|        14|Buffalo Bills|           -6.0|      Giants Stadium|                 52|              10|          null|       1|
|Buffalo Bills|        20|        10|New York Jets|            0.0|Ralph Wilson Stadium|                 68|              13|          null|       2|
|New York Jets|        24|        31|Buffalo Bills|           -4.0|      Giants Stadium|            

In [9]:
NFL_fix_weather = NFL_short.withColumn('weather_detail', F.when(NFL_short.weather_detail=='DOME','1').otherwise('0'))
NFL_fix_weather.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [20]:
NFL_guru_jetsbills = NFL_guru_jetsbills.withColumn('weather_detail', F.when(NFL_short.weather_detail=='DOME','1').otherwise('0'))

In [10]:
NFL_clean = NFL_fix_weather
NFL_clean.show()

+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|           team_home|score_home|score_away|           team_away|spread_favorite|             stadium|weather_temperature|weather_wind_mph|weather_detail|home_win|
+--------------------+----------+----------+--------------------+---------------+--------------------+-------------------+----------------+--------------+--------+
|Tampa Bay Buccaneers|        31|        16|       Detroit Lions|           -3.0|  Houlihan's Stadium|                 79|               9|             0|       2|
|       Buffalo Bills|         7|         9|      Miami Dolphins|           -5.0|Ralph Wilson Stadium|                 74|              15|             0|       1|
|       Chicago Bears|         6|         3|   Green Bay Packers|           -3.0|       Soldier Field|                 78|              11|             0|       2|
|      Denver Br

In [11]:
NFL_clean = NFL_clean.dropna()
NFL_clean_df = NFL_clean.toPandas()
NFL_clean_df.head()

,team_home,score_home,score_away,team_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,Tampa Bay Buccaneers,31,16,Detroit Lions,-3.0,Houlihan's Stadium,79,9,0,2
1,Buffalo Bills,7,9,Miami Dolphins,-5.0,Ralph Wilson Stadium,74,15,0,1
2,Chicago Bears,6,3,Green Bay Packers,-3.0,Soldier Field,78,11,0,2
3,Denver Broncos,10,0,Cincinnati Bengals,-3.0,Mile High Stadium,69,6,0,2
4,Kansas City Chiefs,14,0,Baltimore Colts,-1.0,Arrowhead Stadium,76,8,0,2


In [12]:
results = []
home_teams = NFL_clean.select('team_home').distinct()
home_team_list = home_teams.rdd.map(lambda x: x[0]).collect()
visiting_teams = NFL_clean.select('team_away').distinct()
visiting_team_list = visiting_teams.rdd.map(lambda x: x[0]).collect()
teams_seen = []


In [22]:
save = []
save = pd.DataFrame()
check_df = pd.DataFrame()
all_model_results = pd.DataFrame()

In [35]:
errornum = 1
successnum = 1 
for home_team in home_team_list:
  for visiting_team in visiting_team_list:
    if home_team == visiting_team:
      continue
    else:
      try:
        print (f"Success# {successnum}")
        successnum = successnum+1
        print (f"Teams: H: {home_team}, A: {visiting_team}")
        # Subset your dataframe
        temp_df = NFL_clean.filter(((NFL_clean.team_home == home_team) & (NFL_clean.team_away == visiting_team)) | ((NFL_clean.team_home == visiting_team) & (NFL_clean.team_away == home_team)))
          # Dump it to a pandas df
        temp_df_panda = temp_df.toPandas()
        temp_df_panda = temp_df_panda.drop(['team_home', 'team_away'], axis=1)
        check_df = temp_df_panda
        print (len(temp_df_panda))
        if len(temp_df_panda) > 10:     
          # Get dummies
          stadium_encoded = pd.get_dummies(temp_df_panda, columns=['stadium'])
          save = stadium_encoded
          stadium_encoded.head()
          X = stadium_encoded.drop(columns=['home_win'])
          y = stadium_encoded['home_win']

          # Split into X, Y and training/test sets

          X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1 )#,stratify = y

          # Fit model
          ros = RandomOverSampler(random_state=1)
          X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
          model = LogisticRegression(solver='lbfgs', random_state=1)
          model.fit(X_resampled, y_resampled)

          # Predict
          y_pred = model.predict(X_test)
          accuracy = balanced_accuracy_score(y_test, y_pred)
          matrix = confusion_matrix(y_test, y_pred)

          #results = pd.DataFrame()
          # Put your results in a dataframe

          results['Team_1'] = home_team
          results['Team_2'] = visiting_team
          results['Accuracy'] = accuracy

          # Store results
          #results.append()
          all_model_results.append(results) # = pd.concat([results])
      except Exception as e:
        print(f"Error#: {errornum}, Error Type: {e}")
        print(f"Teams: H: {home_team}, A: {visiting_team}")
        errornum = errornum +1
        

Success# 1
Teams: H: San Francisco 49ers, A: Tennessee Titans
6
Success# 2
Teams: H: San Francisco 49ers, A: Houston Oilers
6
Success# 3
Teams: H: San Francisco 49ers, A: Houston Texans
5
Success# 4
Teams: H: San Francisco 49ers, A: Baltimore Colts
0
Success# 5
Teams: H: San Francisco 49ers, A: Arizona Cardinals
39
Error#: 1, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Arizona Cardinals
Success# 6
Teams: H: San Francisco 49ers, A: Jacksonville Jaguars
5
Success# 7
Teams: H: San Francisco 49ers, A: Tampa Bay Buccaneers
21
Error#: 2, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Tampa Bay Buccaneers
Success# 8
Teams: H: San Francisco 49ers, A: New England Patriots
11
Error#: 3, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: New England Patriots
Success# 9
Teams: H: San Francisco 49ers, A: Kansas City Chiefs
11
Error#: 4, Error Type: list indi

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


20
Error#: 10, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Green Bay Packers
Success# 21
Teams: H: San Francisco 49ers, A: Carolina Panthers
20
Error#: 11, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Carolina Panthers
Success# 22
Teams: H: San Francisco 49ers, A: Denver Broncos
13
Error#: 12, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Denver Broncos
Success# 23
Teams: H: San Francisco 49ers, A: St. Louis Cardinals
5
Success# 24
Teams: H: San Francisco 49ers, A: Dallas Cowboys
23
Error#: 13, Error Type: list indices must be integers or slices, not str
Teams: H: San Francisco 49ers, A: Dallas Cowboys
Success# 25
Teams: H: San Francisco 49ers, A: Los Angeles Chargers
1
Success# 26
Teams: H: San Francisco 49ers, A: Las Vegas Raiders
0
Success# 27
Teams: H: San Francisco 49ers, A: New Orleans Saints
57
Error#: 14, Error Type: list indices 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6
Success# 61
Teams: H: Tennessee Titans, A: Green Bay Packers
5
Success# 62
Teams: H: Tennessee Titans, A: Carolina Panthers
4
Success# 63
Teams: H: Tennessee Titans, A: Denver Broncos
6
Success# 64
Teams: H: Tennessee Titans, A: St. Louis Cardinals
0
Success# 65
Teams: H: Tennessee Titans, A: Dallas Cowboys
6
Success# 66
Teams: H: Tennessee Titans, A: Los Angeles Chargers
1
Success# 67
Teams: H: Tennessee Titans, A: Las Vegas Raiders
0
Success# 68
Teams: H: Tennessee Titans, A: New Orleans Saints
5
Success# 69
Teams: H: Tennessee Titans, A: Washington Redskins
6
Success# 70
Teams: H: Tennessee Titans, A: San Diego Chargers
8
Success# 71
Teams: H: Tennessee Titans, A: Detroit Lions
5
Success# 72
Teams: H: Tennessee Titans, A: Tennessee Oilers
0
Success# 73
Teams: H: Tennessee Titans, A: Los Angeles Rams
2
Success# 74
Teams: H: Tennessee Titans, A: Seattle Seahawks
4
Success# 75
Teams: H: Tennessee Titans, A: Baltimore Ravens
14
Error#: 31, Error Type: list indices must be integers or 

...

Error#: 586, Error Type: list indices must be integers or slices, not str
Teams: H: New York Giants, A: Philadelphia Eagles
Success# 1612
Teams: H: New York Giants, A: Indianapolis Colts
8
Success# 1613
Teams: H: New York Giants, A: Cleveland Browns
8
Success# 1614
Teams: H: New York Giants, A: New York Jets
11
Error#: 587, Error Type: list indices must be integers or slices, not str
Teams: H: New York Giants, A: New York Jets
Success# 1615
Teams: H: New York Giants, A: Miami Dolphins
7
Success# 1616
Teams: H: New York Giants, A: Chicago Bears
14
Error#: 588, Error Type: list indices must be integers or slices, not str
Teams: H: New York Giants, A: Chicago Bears
Success# 1617
Teams: H: New York Giants, A: Phoenix Cardinals
12
Error#: 589, Error Type: list indices must be integers or slices, not str
Teams: H: New York Giants, A: Phoenix Cardinals
Success# 1618
Teams: H: New York Giants, A: Oakland Raiders
8
Success# 1619
Teams: H: New York Giants, A: Atlanta Falcons
18
Error#: 590, Erro

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Error#: 599, Error Type: list indices must be integers or slices, not str
Teams: H: New York Giants, A: Minnesota Vikings
Success# 1637
Teams: H: New York Giants, A: Cincinnati Bengals
8
Success# 1638
Teams: H: New York Giants, A: Los Angeles Raiders
4
Success# 1639
Teams: H: New York Giants, A: St. Louis Rams
10
Success# 1640
Teams: H: New York Giants, A: Washington Football Team
1
Success# 1641
Teams: H: St. Louis Rams, A: San Francisco 49ers
42
Error#: 600, Error Type: list indices must be integers or slices, not str
Teams: H: St. Louis Rams, A: San Francisco 49ers
Success# 1642
Teams: H: St. Louis Rams, A: Tennessee Titans
5
Success# 1643
Teams: H: St. Louis Rams, A: Houston Oilers
0
Success# 1644
Teams: H: St. Louis Rams, A: Houston Texans
3
Success# 1645
Teams: H: St. Louis Rams, A: Baltimore Colts
0
Success# 1646
Teams: H: St. Louis Rams, A: Arizona Cardinals
30
Error#: 601, Error Type: list indices must be integers or slices, not str
Teams: H: St. Louis Rams, A: Arizona Cardina

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


20
Error#: 607, Error Type: list indices must be integers or slices, not str
Teams: H: St. Louis Rams, A: Carolina Panthers
Success# 1663
Teams: H: St. Louis Rams, A: Denver Broncos
6
Success# 1664
Teams: H: St. Louis Rams, A: St. Louis Cardinals
0
Success# 1665
Teams: H: St. Louis Rams, A: Dallas Cowboys
7
Success# 1666
Teams: H: St. Louis Rams, A: Los Angeles Chargers
0
Success# 1667
Teams: H: St. Louis Rams, A: Las Vegas Raiders
0
Success# 1668
Teams: H: St. Louis Rams, A: New Orleans Saints
22
Error#: 608, Error Type: list indices must be integers or slices, not str
Teams: H: St. Louis Rams, A: New Orleans Saints
Success# 1669
Teams: H: St. Louis Rams, A: Washington Redskins
14
Error#: 609, Error Type: list indices must be integers or slices, not str
Teams: H: St. Louis Rams, A: Washington Redskins
Success# 1670
Teams: H: St. Louis Rams, A: San Diego Chargers
5
Success# 1671
Teams: H: St. Louis Rams, A: Detroit Lions
8
Success# 1672
Teams: H: St. Louis Rams, A: Tennessee Oilers
0
S

In [21]:
y

0     1
1     1
2     2
3     2
4     2
5     2
6     1
7     2
8     1
9     2
10    1
11    2
12    2
13    1
14    2
15    1
16    2
17    1
18    2
19    2
20    1
21    2
22    2
23    2
24    1
25    2
26    2
27    2
28    2
29    1
Name: home_win, dtype: int32

In [34]:
all_model_results.head()

""


In [27]:
check_df.head()

,score_home,score_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win


In [ ]:
NFL_guru_jetsbills_df = NFL_guru_jetsbills.toPandas()
NFL_guru_jetsbills_df.head()

,score_home,score_away,spread_favorite,stadium,weather_temperature,weather_wind_mph,weather_detail,home_win
0,46,31,-3.0,Ralph Wilson Stadium,50.0,9.0,0,2
1,12,14,-6.0,Giants Stadium,52.0,10.0,0,1
2,20,10,0.0,Ralph Wilson Stadium,68.0,13.0,0,2
3,24,31,-4.0,Giants Stadium,44.0,12.0,0,1
4,31,0,-4.0,Ralph Wilson Stadium,66.0,8.0,0,2


In [14]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [15]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression


In [ ]:
NFL_jetsbills_encoded = pd.get_dummies(NFL_guru_jetsbills_df, columns=['stadium'])
NFL_jetsbills_encoded.shape

(85, 14)

In [ ]:
NFL_jetsbills_encoded = NFL_jetsbills_encoded.dropna()

In [ ]:
NFL_jetsbills_encoded.shape

(79, 14)

In [ ]:
X = NFL_jetsbills_encoded.drop(['home_win'], axis=1)
y = NFL_jetsbills_encoded['home_win']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)


In [ ]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({1: 31, 2: 31})

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9545454545454546

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 1, 10]])

In [ ]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.90      1.00      0.91      0.95      0.95      0.92         9
          2       1.00      0.91      1.00      0.95      0.95      0.90        11

avg / total       0.96      0.95      0.96      0.95      0.95      0.91        20



In [108]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({1: 31, 2: 31})

In [109]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [110]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9545454545454546

In [111]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 1, 10]])

In [112]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.90      1.00      0.91      0.95      0.95      0.92         9
          2       1.00      0.91      1.00      0.95      0.95      0.90        11

avg / total       0.96      0.95      0.96      0.95      0.95      0.91        20



In [113]:
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({1: 29, 2: 30})

In [115]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [116]:
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [117]:
confusion_matrix(y_test, y_pred)

array([[ 9,  0],
       [ 0, 11]])

In [118]:
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

          1       1.00      1.00      1.00      1.00      1.00      1.00         9
          2       1.00      1.00      1.00      1.00      1.00      1.00        11

avg / total       1.00      1.00      1.00      1.00      1.00      1.00        20

